<a href="https://www.kaggle.com/code/anggoletomi/box-package-size-optimization?scriptVersionId=106100017" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

- Problem Statement : Company have a list of box with different dimensions along with the cost of each box. Current situation the operational cost for box has become the pain point. Company wants to know if their current box size options are already correct and fit for each orders, so they want the model to decide box size for each orders based on their sales orders history. Company also want to know in an ideal situation how much is the average cost per box should company spend.

- Dataset : Order Details, SKU Dimensions, Box Dimensions

- Goal & Objective : Find the most fit box size for each orders, on which box size should company spend more, so they can minimize the cost for unused box, and choose the correct size for each orders in the future.

In [107]:
## import dependencies

import itertools
import pandas as pd
from tqdm import tqdm
from py3dbp import Packer, Bin, Item

## 1. Get the 'Box Dimensions' dataframe

In [108]:
## Define Maximum Weight or Volume that the box can contains

MAX_WEIGHT = 10 ** 9

In [109]:
## create list with loop

box_dimension_df = pd.read_csv("/kaggle/input/uq-retail-order/uq_box_dimensions.csv")

## check the dataframe info

print(f'There are {len(box_dimension_df)} list of box size.')
print()

box_dimension_df.info()

There are 33 list of box size.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33 entries, 0 to 32
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             33 non-null     int64  
 1   box_number     33 non-null     object 
 2   length         33 non-null     float64
 3   width          33 non-null     float64
 4   height         33 non-null     float64
 5   box_dimension  33 non-null     object 
 6   cost_per_box   33 non-null     int64  
dtypes: float64(3), int64(2), object(2)
memory usage: 1.9+ KB


In [110]:
# show data sample to quick take a look on the datasets

box_dimension_df.head(5)

,id,box_number,length,width,height,box_dimension,cost_per_box
0,1,BOX-1,41.1,34.0,22.4,41.1 x 34 x 22.4,7800
1,2,BOX-2,6.0,6.0,11.0,6 x 6 x 11,2000
2,3,BOX-3,12.0,11.5,5.0,12 x 11.5 x 5,2400
3,4,BOX-4,20.0,10.0,8.5,20 x 10 x 8.5,3200
4,5,BOX-5,10.5,4.3,2.0,10.5 x 4.3 x 2,1800


In [111]:
## MAP COST & DIMENSION

## this we will use later on the final dataframe to lookup on cost and dimension

lookup_cost = dict(zip(box_dimension_df['box_number'].str.upper(), box_dimension_df['cost_per_box']))
lookup_dimension = dict(zip(box_dimension_df['box_number'].str.upper(), box_dimension_df['box_dimension']))

## 2. Remove possibility of duplicate box size

Such as 10 x 6 x 5 = 10 x 5 x 6 = 5 x 10 x 6 = etc..

In [112]:
sort_first = box_dimension_df['box_dimension'].tolist()

new_sorted = []

for sr in sort_first:
    new_str_list = ''.join(sorted(sr))
    new_sorted.append(new_str_list)

box_dimension_df['new_sorted'] = new_sorted

print(f'total row before : {len(box_dimension_df)}')

box_dimension_df = box_dimension_df.drop_duplicates(subset = ['new_sorted'] , keep = 'first')
box_dimension_df.drop(['new_sorted'], inplace=True, axis=1)
      
print(f'total row after remove duplicate : {len(box_dimension_df)}')

total row before : 33
total row after remove duplicate : 33


There is no duplicate on our datasets.

## 3. Get the 'SKU Dimensions' dataframe

In [113]:
sku_dimension_df = pd.read_csv("/kaggle/input/uq-retail-order/sku_dimensions.csv")

## take a quick look on the dataset

sku_dimension_df.head(5) 

,sku_id,length,width,height,sku_dimension
0,UQ001,14.0,9.0,1.0,14 x 9 x 1
1,UQ002,9.0,8.3,20.5,9 x 8.3 x 20.5
2,UQ003,7.0,7.0,14.0,7 x 7 x 14
3,UQ004,7.0,7.0,15.0,7 x 7 x 15
4,UQ005,9.0,9.0,9.0,9 x 9 x 9


In [114]:
## check the dataset information

sku_dimension_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 349 entries, 0 to 348
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   sku_id         349 non-null    object 
 1   length         349 non-null    float64
 2   width          349 non-null    float64
 3   height         349 non-null    float64
 4   sku_dimension  349 non-null    object 
dtypes: float64(3), object(2)
memory usage: 13.8+ KB


This datasets already clean, so there is no need to check on duplicate value or null values

## 3. Get the 'Order Lists' dataframe

In [115]:
order_sku_df = pd.read_csv("/kaggle/input/uq-retail-order/uq_orders.csv")

sku_dimension_df.head(5)

,sku_id,length,width,height,sku_dimension
0,UQ001,14.0,9.0,1.0,14 x 9 x 1
1,UQ002,9.0,8.3,20.5,9 x 8.3 x 20.5
2,UQ003,7.0,7.0,14.0,7 x 7 x 14
3,UQ004,7.0,7.0,15.0,7 x 7 x 15
4,UQ005,9.0,9.0,9.0,9 x 9 x 9


In [116]:
sku_dimension_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 349 entries, 0 to 348
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   sku_id         349 non-null    object 
 1   length         349 non-null    float64
 2   width          349 non-null    float64
 3   height         349 non-null    float64
 4   sku_dimension  349 non-null    object 
dtypes: float64(3), object(2)
memory usage: 13.8+ KB


This datasets already clean, so there is no need to check on duplicate value or null values

## 4. Calculate the Box Size

In [117]:
## Search the volume of each box

box_dimension_df['volume'] = box_dimension_df['length'] * box_dimension_df['width'] * box_dimension_df['height']
box_dimension_df.sort_values(['volume', 'id'], inplace=True)
box_dimension_df.drop_duplicates('volume', inplace=True)

box_dimension_df.head(3)

,id,box_number,length,width,height,box_dimension,cost_per_box,volume
6,7,BOX-7,2.6,2.6,8.4,2.6 x 2.6 x 8.4,1600,56.784
4,5,BOX-5,10.5,4.3,2.0,10.5 x 4.3 x 2,1800,90.300
1,2,BOX-2,6.0,6.0,11.0,6 x 6 x 11,2000,396.000


In [118]:
## Merge order details and sku dimensions dataframe

df = order_sku_df.copy()
df = df.merge(sku_dimension_df, how='left', on='sku_id')

df = df.dropna() ## drop null values if any

df.head(3)

,order_number,sku_id,quantity,length,width,height,sku_dimension
0,SJK121090605204,UQ166,2,15.5,2.5,4.0,15.5 x 2.5 x 4
1,SJK121050600283,UQ002,1,9.0,8.3,20.5,9 x 8.3 x 20.5
2,SJK121072610089,UQ114,1,9.0,6.0,23.0,9 x 6 x 23


In [119]:
## Create the function

box_numbers = box_dimension_df['box_number'].values
xs = box_dimension_df['length'].values
ys = box_dimension_df['width'].values
zs = box_dimension_df['height'].values

def solve(order_df: pd.DataFrame):
    for box_number, x, y, z in zip(box_numbers, xs, ys, zs):
        packer = Packer()
        packer.add_bin(Bin(box_number, x, y, z, MAX_WEIGHT))

        for _, row in order_df.iterrows():
            for _ in range(row['quantity']):
                packer.add_item(Item(row['sku_id'], row['length'], row['width'], row['height'], 1))

        packer.pack()

        for b in packer.bins:
            if len(b.unfitted_items) == 0:
                return box_number
            
    return 'UNFITTED'

In [120]:
ddf = df.copy()

res_matrix = []
for order_number, order_df in tqdm(ddf.groupby('order_number')):
    res_matrix.append([order_number, solve(order_df)])
    
res_df = pd.DataFrame(data=res_matrix, columns=['order_number', 'box_number'])

100%|██████████| 998/998 [00:03<00:00, 284.15it/s]


## 5. Result Summary

In [121]:
## CLEAN FINAL DATAFRAME

to_basket_df = order_sku_df.copy()

to_basket_df['total_basket_sku'] = to_basket_df.groupby(['order_number'])['sku_id'].transform('nunique')
to_basket_df['total_basket_pcs'] = to_basket_df.groupby(['order_number'])['quantity'].transform('sum')

basket_df = to_basket_df[['order_number','total_basket_sku','total_basket_pcs']]
basket_df.columns = ['order_number','total_sku','total_qty']
basket_df = basket_df.drop_duplicates()

In [122]:
final_df = res_df.merge(box_dimension_df[['box_number','box_dimension']], how='left', on='box_number')
final_df = final_df.merge(basket_df, how='left', on='order_number')

final_df = final_df.groupby(['box_number']).agg({'order_number' : ['count']}).reset_index()

final_df.columns = ['box_number','total_order']

final_df = final_df.sort_values(['total_order'],ascending=False)

final_df['all_order'] = final_df['total_order'].sum()
final_df['perc_of_order'] = round((final_df['total_order'] / final_df['all_order']) * 100,1)

top10_box = final_df.head(10)
top10_box = top10_box['perc_of_order'].sum()
top10_box = str(round(top10_df,2)) + '%'

final_df['perc_of_order'] = final_df['perc_of_order'].astype(str) + '%'

final_df.drop(['all_order'], inplace=True, axis=1)

## GET COST & DIMENSIONS

final_df['total_box_cost'] = final_df['box_number'].map(lookup_cost)
final_df['total_box_cost'] = final_df['total_box_cost'] * final_df['total_order']

final_df['box_dimension'] = final_df['box_number'].map(lookup_dimension)

In [123]:
avg_df = final_df.dropna()

avg_cost_per_box = avg_df['total_box_cost'].sum() / avg_df['total_order'].sum()
avg_cost_per_box = round(avg_cost_per_box,0)

print()
print(f'average cost per box , exclude UNFITTED : {avg_cost_per_box} IDR')
print(f'Top 10 box out of {len(box_dimension_df)} list of box already can covered : {top10_box} of orders')
print()

final_df.head(10)


average cost per box , exclude UNFITTED : 3437.0 IDR
Top 10 box out of 33 list of box already can covered : 85.6% of orders



,box_number,total_order,perc_of_order,total_box_cost,box_dimension
15,BOX-27,238,23.8%,714000.0,22 x 10 x 7
19,BOX-30,187,18.7%,710600.0,25 x 15 x 8
17,BOX-29,122,12.2%,268400.0,18 x 7 x 5
3,BOX-14,74,7.4%,266400.0,19 x 15 x 10
20,BOX-31,55,5.5%,264000.0,30 x 19 x 10
2,BOX-13,45,4.5%,117000.0,15 x 12 x 5
25,BOX-8,45,4.5%,153000.0,26 x 21.5 x 4
18,BOX-3,34,3.4%,81600.0,12 x 11.5 x 5
10,BOX-22,34,3.4%,170000.0,25 x 18 x 15
24,BOX-6,22,2.2%,96800.0,25 x 12 x 16
